# 1.  Imports and Data

In [1]:
from sklearn.decomposition import PCA
import matplotlib as mpl 
import numpy as np
import os
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler



from utils import stratified_train_test_group_kfold
from utils import pred2class
from utils import model_test_regression
from regression_codes import gpr_gridsearch



with open("Data/MoS2_Analysis_Data_trained2", "rb") as fp:   # Unpickling
    MoS2_Proj1_Class_Data = pickle.load(fp)

#features = MoS2_ImageNet_100_data[0]
df = pd.DataFrame(MoS2_Proj1_Class_Data)
print(df.keys())
T_target = np.array(list(df['T']))
features = np.array(list(df['ImageNet']))
sampleId = np.array(list(df['sampleId']))
train_val_sampleId, test_sampleId, train_val_Y, test_Y = train_test_split(sampleId, T_target, test_size=0.1, stratify = T_target, random_state=40)

0.10.1


# 2. Data Splitting and Scaling

In [2]:


X = features
Y = np.array(T_target)
groups = np.array(sampleId)




train_val_groups, train_val_X, train_val_Y, test_X, test_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=0)
train_val_Y = train_val_Y.flatten()
#val_Y = val_Y.flatten()
test_Y = test_Y.flatten()

print(train_val_X.shape)
print(train_val_Y.shape)
print(test_X.shape)
print(test_Y.shape)


oversample = RandomOverSampler(sampling_strategy='not majority')

scaler = StandardScaler().fit(train_val_X)
train_val_X = scaler.transform(train_val_X)
test_X = scaler.transform(test_X)

Index(['sampleId', 'sampleLabel', 'image', 'T', 'ImageNet', 'MicroNet'], dtype='object')


In [6]:
! mkdir -p 'Model/Reg/TrainedImageNet/Up'

# 3. Runing Cross-Validation

In [6]:
N_restarts_optimizer = [i for i in range(12)]
Normalize_y = [True, False]
Alpha = [1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10, 1e-12]


def cross_10_folds_gpc(train_val_X, train_val_Y):
    
    best_train = []
    best_val = []
    best_variables = []
    for fold in range(10):
        group, train_X, train_Y, val_X, val_Y = stratified_train_test_group_kfold(train_val_X, train_val_Y, train_val_groups, n_splits=10, test_fold=fold)
        
        model_path = f'regression/ImageNet/aug3/GPR_{fold}_model.sav'
        
        train_X, train_Y = oversample.fit_resample(train_X, train_Y)
        variables_best, performance_best =gpr_gridsearch(train_X, train_Y, val_X, val_Y,Alpha, N_restarts_optimizer, Normalize_y, fold, model_path)
        best_train.append(performance_best['accuracy_train'])
        best_val.append(performance_best['accuracy_val'])
        best_variables.append(variables_best)
        print(f'fold: {fold} done!')
    return best_train, best_val, best_variables


best_train, best_val, best_variables = cross_10_folds_gpc(train_val_X, train_val_Y)




print(best_train)
print(best_val)
print(f'train_acc_mean: {np.mean(best_train) :.3f}, std: {np.std(best_train) :.3f}, val_acc_mean: {np.mean(best_val):.3f}, std: {np.std(best_val):.3f}')

<class 'generator'>
best_train_acc: 1.0, best_val_acc: 0.5
fold: 0 done!
<class 'generator'>
best_train_acc: 1.0, best_val_acc: 0.4583333333333333
fold: 1 done!
<class 'generator'>
best_train_acc: 1.0, best_val_acc: 0.4583333333333333
fold: 2 done!
<class 'generator'>
best_train_acc: 1.0, best_val_acc: 0.5
fold: 3 done!
<class 'generator'>
best_train_acc: 1.0, best_val_acc: 0.5
fold: 4 done!
<class 'generator'>
best_train_acc: 1.0, best_val_acc: 0.5217391304347826
fold: 5 done!
<class 'generator'>
best_train_acc: 1.0, best_val_acc: 0.5217391304347826
fold: 6 done!
<class 'generator'>
best_train_acc: 1.0, best_val_acc: 0.5217391304347826
fold: 7 done!
<class 'generator'>
best_train_acc: 1.0, best_val_acc: 0.5217391304347826
fold: 8 done!
<class 'generator'>
best_train_acc: 1.0, best_val_acc: 0.4782608695652174
fold: 9 done!
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[0.5, 0.4583333333333333, 0.4583333333333333, 0.5, 0.5, 0.5217391304347826, 0.5217391304347826, 0.5217391304347826

# 4. Model Testing

In [5]:
best_fold = 5

model_path = f'regression/ImageNet/aug3/GPR_{best_fold}_model.sav'
# Fitting different train in the same hyperparameters
best_test, root_mean_squared_error, confusion_matrix_test = model_test_regression(train_val_X, train_val_Y, train_val_groups,test_X, test_Y, best_fold, model_path)

print('best_test: ', best_test)
print('rmse: ', root_mean_squared_error)
print(f'test_acc_mean: {np.mean(best_test) :.3f}, std: {np.std(best_test) :.3f}, rmse_mean: {np.mean(root_mean_squared_error)}, rmse_std: {np.std(root_mean_squared_error)}')

print("......")

print('5th confusion matrix: ', [list(item) for item in confusion_matrix_test[4]])

<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
best_test:  [0.48148148148148145, 0.48148148148148145, 0.48148148148148145, 0.48148148148148145, 0.48148148148148145, 0.48148148148148145, 0.48148148148148145, 0.48148148148148145, 0.48148148148148145, 0.48148148148148145]
rmse:  [36.004111547468455, 36.00411158317267, 36.00411187080335, 36.00411154820703, 36.00411154783775, 36.00411158112145, 36.004111549897345, 36.00411154768395, 36.00411448472109, 36.004111695785014]
test_acc_mean: 0.481, std: 0.000, rmse_mean: 36.00411189566981, rmse_std: 8.685602119108991e-07
......
5th confusion matrix:  [[0, 3, 0], [0, 13, 0], [0, 11, 0]]


In [4]:
best_fold = 5
#PATH = os.path.join('Model', model_path)
PATH = f'Model/Reg/TrainedImageNet/Up/GPR_{best_fold}_model.sav'
loaded_model = pickle.load(open(PATH, 'rb'))
loaded_model.get_params()

{'alpha': 0.0001,
 'copy_X_train': True,
 'kernel': None,
 'n_restarts_optimizer': 0,
 'normalize_y': True,
 'optimizer': 'fmin_l_bfgs_b',
 'random_state': 1}